<a href="https://colab.research.google.com/github/Aryanupadhyay23/Machine-Learning-/blob/main/optuna_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
# !pip install optuna

In [87]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = [
    'Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness',
    'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'
]

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [88]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [89]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")


Training set shape: (537, 8)
Test set shape: (231, 8)


In [90]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(
        model,
        X_train,
        y_train,
        cv=3,
        scoring='accuracy'
    ).mean()

    # Return the accuracy score for Optuna to maximize
    return score


In [91]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize')  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters


[I 2026-02-07 15:07:03,586] A new study created in memory with name: no-name-0aa43233-b2d3-4377-9bcf-9fe230b12426
[I 2026-02-07 15:07:04,349] Trial 0 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 134, 'max_depth': 5, 'min_samples_split': 3}. Best is trial 0 with value: 0.7672253258845437.
[I 2026-02-07 15:07:04,652] Trial 1 finished with value: 0.7597765363128491 and parameters: {'n_estimators': 50, 'max_depth': 9, 'min_samples_split': 5}. Best is trial 0 with value: 0.7672253258845437.
[I 2026-02-07 15:07:05,483] Trial 2 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 141, 'max_depth': 14, 'min_samples_split': 10}. Best is trial 0 with value: 0.7672253258845437.
[I 2026-02-07 15:07:06,034] Trial 3 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 90, 'max_depth': 18, 'min_samples_split': 7}. Best is trial 0 with value: 0.7672253258845437.
[I 2026-02-07 15:07:06,414] Trial 4 finished with value: 0.76163873370577

In [92]:
# Print the best result
print(f"Best trial accuracy: {study.best_trial.value}")
print(f"Best hyperparameters: {study.best_trial.params}")


Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 121, 'max_depth': 19, 'min_samples_split': 2}


In [93]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(
    **study.best_trial.params,
    random_state=42
)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f"Test Accuracy with best hyperparameters: {test_accuracy:.2f}")


Test Accuracy with best hyperparameters: 0.74


In [94]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(
        model,
        X_train,
        y_train,
        cv=3,
        scoring='accuracy'
    ).mean()

    # Return the accuracy score for Optuna to maximize
    return score


In [95]:
study = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.RandomSampler()
)  # We aim to maximize accuracy

study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters


[I 2026-02-07 15:07:46,083] A new study created in memory with name: no-name-a5879f5b-56a6-4309-866f-020bcbf0f3a6
[I 2026-02-07 15:07:47,227] Trial 0 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 197, 'max_depth': 7, 'min_samples_split': 2}. Best is trial 0 with value: 0.7672253258845437.
[I 2026-02-07 15:07:48,272] Trial 1 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 180, 'max_depth': 7, 'min_samples_split': 3}. Best is trial 1 with value: 0.7728119180633147.
[I 2026-02-07 15:07:49,258] Trial 2 finished with value: 0.7616387337057727 and parameters: {'n_estimators': 168, 'max_depth': 8, 'min_samples_split': 4}. Best is trial 1 with value: 0.7728119180633147.
[I 2026-02-07 15:07:50,565] Trial 3 finished with value: 0.7579143389199254 and parameters: {'n_estimators': 194, 'max_depth': 3, 'min_samples_split': 5}. Best is trial 1 with value: 0.7728119180633147.
[I 2026-02-07 15:07:51,161] Trial 4 finished with value: 0.759776536312849

In [96]:
# Print the best result
print(f"Best trial accuracy: {study.best_trial.value}")
print(f"Best hyperparameters: {study.best_trial.params}")


Best trial accuracy: 0.7765363128491619
Best hyperparameters: {'n_estimators': 134, 'max_depth': 15, 'min_samples_split': 2}


In [97]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(
    **study.best_trial.params,
    random_state=42
)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f"Test Accuracy with best hyperparameters: {test_accuracy:.2f}")


Test Accuracy with best hyperparameters: 0.75


In [98]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20],
    'min_samples_split': [2, 5, 10]
}

In [99]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.GridSampler(search_space)
)

study.optimize(objective)

[I 2026-02-07 15:08:26,090] A new study created in memory with name: no-name-13e02c41-c22b-4f1a-9307-6dbecd7f6eff
[I 2026-02-07 15:08:26,703] Trial 0 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15, 'min_samples_split': 5}. Best is trial 0 with value: 0.7653631284916201.
[I 2026-02-07 15:08:26,994] Trial 1 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 50, 'max_depth': 5, 'min_samples_split': 5}. Best is trial 0 with value: 0.7653631284916201.
[I 2026-02-07 15:08:28,028] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 150, 'max_depth': 15, 'min_samples_split': 2}. Best is trial 2 with value: 0.7728119180633147.
[I 2026-02-07 15:08:29,344] Trial 3 finished with value: 0.7616387337057727 and parameters: {'n_estimators': 150, 'max_depth': 15, 'min_samples_split': 5}. Best is trial 2 with value: 0.7728119180633147.
[I 2026-02-07 15:08:29,829] Trial 4 finished with value: 0.7672253258845

In [100]:
# Print the best result
print(f"Best trial accuracy: {study.best_trial.value}")
print(f"Best hyperparameters: {study.best_trial.params}")

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5, 'min_samples_split': 2}


In [101]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(
    **study.best_trial.params,
    random_state=42
)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f"Test Accuracy with best hyperparameters: {test_accuracy:.2f}")


Test Accuracy with best hyperparameters: 0.74


In [102]:
# !pip install cmaes

In [103]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.CmaEsSampler()
)

study.optimize(objective, n_trials=50)


[I 2026-02-07 15:09:04,283] A new study created in memory with name: no-name-b43db992-9619-4772-81f2-175670447cc0
[I 2026-02-07 15:09:05,274] Trial 0 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 182, 'max_depth': 4, 'min_samples_split': 3}. Best is trial 0 with value: 0.7635009310986964.
[I 2026-02-07 15:09:06,445] Trial 1 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 153, 'max_depth': 11, 'min_samples_split': 6}. Best is trial 1 with value: 0.7653631284916201.
[I 2026-02-07 15:09:07,519] Trial 2 finished with value: 0.7597765363128491 and parameters: {'n_estimators': 120, 'max_depth': 13, 'min_samples_split': 5}. Best is trial 1 with value: 0.7653631284916201.
[I 2026-02-07 15:09:08,307] Trial 3 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 83, 'max_depth': 15, 'min_samples_split': 7}. Best is trial 3 with value: 0.7672253258845437.
[I 2026-02-07 15:09:09,093] Trial 4 finished with value: 0.7765363128491

In [104]:
# Print the best result
print(f"Best trial accuracy: {study.best_trial.value}")
print(f"Best hyperparameters: {study.best_trial.params}")

Best trial accuracy: 0.7765363128491619
Best hyperparameters: {'n_estimators': 133, 'max_depth': 8, 'min_samples_split': 8}


In [105]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(
    **study.best_trial.params,
    random_state=42
)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f"Test Accuracy with best hyperparameters: {test_accuracy:.2f}")

Test Accuracy with best hyperparameters: 0.75


In [106]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_max_depth,params_min_samples_split,params_n_estimators,system_attrs_cma:generation,system_attrs_cma:optimizer:0,system_attrs_cma:optimizer:1,state
0,0,0.763501,2026-02-07 15:09:04.285097,2026-02-07 15:09:05.274758,0 days 00:00:00.989661,4,3,182,NaN,NaN,NaN,COMPLETE
1,1,0.765363,2026-02-07 15:09:05.275948,2026-02-07 15:09:06.445441,0 days 00:00:01.169493,11,6,153,0.0,NaN,NaN,COMPLETE
2,2,0.759777,2026-02-07 15:09:06.446809,2026-02-07 15:09:07.519408,0 days 00:00:01.072599,13,5,120,0.0,NaN,NaN,COMPLETE
3,3,0.767225,2026-02-07 15:09:07.520807,2026-02-07 15:09:08.307609,0 days 00:00:00.786802,15,7,83,0.0,NaN,NaN,COMPLETE
4,4,0.776536,2026-02-07 15:09:08.309745,2026-02-07 15:09:09.093023,0 days 00:00:00.783278,8,8,133,0.0,NaN,NaN,COMPLETE
5,5,0.774674,2026-02-07 15:09:09.094216,2026-02-07 15:09:09.903824,0 days 00:00:00.809608,12,7,137,0.0,NaN,NaN,COMPLETE
6,6,0.770950,2026-02-07 15:09:09.905074,2026-02-07 15:09:10.894250,0 days 00:00:00.989176,10,6,167,0.0,NaN,NaN,COMPLETE
7,7,0.759777,2026-02-07 15:09:10.895405,2026-02-07 15:09:11.552555,0 days 00:00:00.657150,12,7,106,0.0,NaN,NaN,COMPLETE
8,8,0.757914,2026-02-07 15:09:11.553725,2026-02-07 15:09:12.495919,0 days 00:00:00.942194,8,5,161,1.0,800495f6060000000000008c0a636d6165732e5f636d61...,294284b014b034b0186946811894318000000000000000...,COMPLETE
9,9,0.759777,2026-02-07 15:09:12.497137,2026-02-07 15:09:13.302989,0 days 00:00:00.805852,10,7,138,1.0,NaN,NaN,COMPLETE


In [107]:
study.sampler

In [108]:
study.pruner

In [109]:
# For visualizations
from optuna.visualization import (
    plot_optimization_history,
    plot_parallel_coordinate,
    plot_slice,
    plot_contour,
    plot_param_importances
)

# 1. Optimization History
plot_optimization_history(study).show()


In [110]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()


In [111]:
# 3. Slice Plot
plot_slice(study).show()

In [112]:
# 4. Contour Plot
plot_contour(study).show()

In [113]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [114]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.svm import SVC

In [115]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical(
        'classifier',
        ['SVM', 'RandomForest', 'GradientBoosting']
    )

    # ---------------- SVM ----------------
    if classifier_name == 'SVM':
        C = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical(
            'kernel',
            ['linear', 'rbf', 'poly', 'sigmoid']
        )
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(
            C=C,
            kernel=kernel,
            gamma=gamma,
            random_state=42
        )

    # ---------------- Random Forest ----------------
    elif classifier_name == 'RandomForest':
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    # ---------------- Gradient Boosting ----------------
    elif classifier_name == 'GradientBoosting':
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float(
            'learning_rate', 0.01, 0.3, log=True
        )
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return mean accuracy
    score = cross_val_score(
        model,
        X_train,
        y_train,
        cv=3,
        scoring='accuracy'
    ).mean()

    return score

In [116]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2026-02-07 15:09:42,677] A new study created in memory with name: no-name-d125e0fd-39a0-4fe8-83d3-9f7a9a208c48
[I 2026-02-07 15:09:44,129] Trial 0 finished with value: 0.7802607076350093 and parameters: {'classifier': 'RandomForest', 'n_estimators': 282, 'max_depth': 6, 'min_samples_split': 2, 'min_samples_leaf': 8, 'bootstrap': False}. Best is trial 0 with value: 0.7802607076350093.
[I 2026-02-07 15:09:46,367] Trial 1 finished with value: 0.7635009310986964 and parameters: {'classifier': 'RandomForest', 'n_estimators': 255, 'max_depth': 11, 'min_samples_split': 8, 'min_samples_leaf': 1, 'bootstrap': True}. Best is trial 0 with value: 0.7802607076350093.
[I 2026-02-07 15:09:46,397] Trial 2 finished with value: 0.7821229050279329 and parameters: {'classifier': 'SVM', 'C': 0.3820540155442586, 'kernel': 'linear', 'gamma': 'scale'}. Best is trial 2 with value: 0.7821229050279329.
[I 2026-02-07 15:09:47,930] Trial 3 finished with value: 0.7541899441340782 and parameters: {'classifier': '

In [117]:
# Retrieve the best trial
best_trial = study.best_trial

print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.11142343790498098, 'kernel': 'linear', 'gamma': 'scale'}
Best trial accuracy: 0.7895716945996275


In [118]:
# View all trials as a DataFrame
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.780261,2026-02-07 15:09:42.681864,2026-02-07 15:09:44.129088,0 days 00:00:01.447224,NaN,False,RandomForest,NaN,NaN,NaN,6.0,8.0,2.0,282.0,COMPLETE
1,1,0.763501,2026-02-07 15:09:44.130714,2026-02-07 15:09:46.367296,0 days 00:00:02.236582,NaN,True,RandomForest,NaN,NaN,NaN,11.0,1.0,8.0,255.0,COMPLETE
2,2,0.782123,2026-02-07 15:09:46.368495,2026-02-07 15:09:46.397205,0 days 00:00:00.028710,0.382054,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
3,3,0.754190,2026-02-07 15:09:46.398257,2026-02-07 15:09:47.930065,0 days 00:00:01.531808,NaN,True,RandomForest,NaN,NaN,NaN,14.0,9.0,5.0,284.0,COMPLETE
4,4,0.743017,2026-02-07 15:09:47.931351,2026-02-07 15:09:50.618428,0 days 00:00:02.687077,NaN,NaN,GradientBoosting,NaN,NaN,0.108682,11.0,3.0,5.0,142.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.785847,2026-02-07 15:10:20.374619,2026-02-07 15:10:20.406601,0 days 00:00:00.031982,0.321453,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.789572,2026-02-07 15:10:20.408125,2026-02-07 15:10:20.441425,0 days 00:00:00.033300,0.154713,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.754190,2026-02-07 15:10:20.442796,2026-02-07 15:10:21.761443,0 days 00:00:01.318647,NaN,True,RandomForest,NaN,NaN,NaN,5.0,8.0,3.0,228.0,COMPLETE
98,98,0.789572,2026-02-07 15:10:21.762863,2026-02-07 15:10:21.814189,0 days 00:00:00.051326,0.113336,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [119]:
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
SVM,78
RandomForest,12
GradientBoosting,10


In [120]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
GradientBoosting,0.743762
RandomForest,0.765984
SVM,0.778184


In [121]:
# 1. Optimization History
plot_optimization_history(study).show()

In [122]:
# 2. Slice Plot
plot_slice(study).show()

In [123]:
# 3. Hyperparameter Importance
plot_param_importances(study).show()